# UHFQA

Just like the driver for the HDAWG in the previous example, we now use the `tk.UHFQA` instrument driver.

In [1]:
import zhinst.toolkit as tk

uhfqa = tk.UHFQA("qa1", "dev2496", interface="1gbe", host="10.42.0.226")
uhfqa.setup()           # set up data server connection
uhfqa.connect_device()  # connect device to data server

Successfully connected to data server at 10.42.0.2268004 api version: 6
Successfully connected to device DEV2496 on interface 1GBE


## AWG core of the UHFQA

Also the UHFQA features one *AWG Core*. 

In [2]:
uhfqa.awg

qa1: <zhinst.toolkit.control.drivers.uhfqa.AWG object at 0x000001ACE7967048>
    parent  : <zhinst.toolkit.control.drivers.uhfqa.UHFQA object at 0x000001ACE63305C8>
    index   : 0
    sequence: 
           type: SequenceType.NONE
            ('target', <DeviceTypes.UHFQA: 'uhfqa'>)
            ('clock_rate', 2400000000.0)
            ('period', 0.0001)
            ('trigger_mode', <TriggerMode.SEND_TRIGGER: 'Send Trigger'>)
            ('repetitions', 1)
            ('alignment', <Alignment.END_WITH_TRIGGER: 'End with Trigger'>)
            ('n_HW_loop', 1)
            ('dead_time', 5e-06)
            ('trigger_delay', 0)
            ('latency', 1.6e-07)
            ('trigger_cmd_1', 'setTrigger(1);\n')
            ('trigger_cmd_2', 'setTrigger(0);\n')
            ('wait_cycles', 0)
            ('dead_cycles', 1500)
            ('reset_phase', False)

## Readout channels of the UHFQA

The UHFQA comes with signal processing streams for up to ten channels in parallel. The settings for the readout are grouped by channel in a list of all ten `channels`. Each item in the `channels` property of the UHFQA is one `ReadoutChannel` object that represent the signal processing path for one of the ten channels.

In [3]:
uhfqa.channels[0]

Readout Channel 0:  <zhinst.toolkit.control.drivers.uhfqa.ReadoutChannel object at 0x000001ACE7967288>
     rotation          : 0.0
     threshold         : 0.0
      Weighted Integration Disabled

Each of the channels follows the following signal processing steps:

1. Demodulation of the input signal
2. Rotation in the complex plane
3. Thresholding for binary result values


The values for the rotation and thresholding stages can be set using the `rotation` and `threshold` parameter of the *channel*. 

The standard mode for the demodulation of input signals is the *weighted integration* mode. This corresponds to setting the integration weights for the two quadratures of the input signal to oscillate at a given demodulation frequency. When enabling the weighted integration with `ch.enable()`, the integration weights for the two quadratures are set. The demodulation frequency is set to the parameter `readout_frequency`.

Enabling weighted integration for the first four channels of the UHFQA and setting their demodulation frequency could look like this: 

In [4]:
freqs = [85.6e6, 101.3e6, 132.8e6]

for ch in uhfqa.channels[:3]:
    ch.enable()
    ch.readout_frequency(freqs[ch.index])

The resut vector of each channel can be retrieved from the instrument by calling the read-only parameter *result*.

In [5]:
uhfqa.channels[0].result

Node: qas/0/result/data/0/wave
Description: Returns the result vector of the readout channel.
Type: Numpy array
Properties: Read
Unit: None

In [6]:
uhfqa.channels[0].result()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Readout parameters

There are readout parameters that are not specific to one single channel but affect all ten readout channels. These are

* the `integration_time`: the time in seconds used for integrating the input signals
* the `result_source` lets the user select at which point in the signal processing path the `result` value should be taken
* the `averaging_mode` specifies if the hardware averages on the device should be taken in a *sequential* or *cyclic* way 
* the `crosstalk_matrix` specifies a 10 x 10 matrix that can be calibrated to compensate for crosstalk betweeen the channels

These three *parameters* are attributes of the UHFQA instrument driver.

In [7]:
uhfqa.integration_time

Node: qas/0/integration/length
Description: The integration time of the QA Integration unit.
Type: Double
Properties: Read, Write
Unit: s
Value: 2e-06

In [8]:
uhfqa.result_source

Node: qas/0/result/source
Description: The signal source for the QA Results unit. One of {'Crosstalk', 'Threshold', 'Rotation', 'Crosstalk Correlation', 'Threshold Correlation', 'Integration'}.
Type: Integer
Properties: Read, Write
Mapping: {0: 'Crosstalk', 1: 'Threshold', 2: 'Rotation', 4: 'Crosstalk Correlation', 5: 'Threshold Correlation', 7: 'Integration'}
Unit: None

In [9]:
uhfqa.averaging_mode

Node: qas/0/result/mode
Description: Selects the order of the result logger. One of {'Cyclic', 'Sequential'}.
Type: Integer
Properties: Read, Write
Mapping: {0: 'Cyclic', 1: 'Sequential'}
Unit: None

In [10]:
uhfqa.crosstalk_matrix()

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

Other important readout parameters can be accessed through the *nodetree*, for example the 

* *result length*: the number of points to acquire
* *result averages*: the number of hardware averages

In [11]:
uhfqa.nodetree.qa.result.length

Node: /DEV2496/QAS/0/RESULT/LENGTH
Description: The time duration of each capture in samples. A maximum of 4096 samples can be captured, which corresponds to 2.3 µs.
Type: Integer (64 bit)
Properties: Read, Write, Setting
Unit: None

In [12]:
uhfqa.nodetree.qa.result.averages

Node: /DEV2496/QAS/0/RESULT/AVERAGES
Description: log2 of the number of averages to perform, i.e. 0 means no averaging, 1 means 2 values are averaged, etc. Maximum setting is 15 meaning 2^15 values are averaged.
Type: Integer (64 bit)
Properties: Read, Write, Setting
Unit: None

## *Arm* the UHFQA readout

The `arm(...)` method of the UHFQA prepares the device for data acquisition. It enables the *Results Acquisition* and resets the acquired points to zero. This should be done before every measurement. The method also includes a shortcut to setting the values *result length* and *result averages*. They can be specified as keyword arguments. If the keyword arguemnts are not specified, nothing is changed.

In [13]:
uhfqa.arm(length=1e3, averages=2**5)